# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv file from previous analysis
readcsv = pd.read_csv("Output/weather_df.csv")
readcsv.head()

del readcsv["Unnamed: 0"]

readcsv.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,nyurba,11,RU,1604279755,90,63.28,118.33,1.98,9.89
1,majene,100,ID,1604279755,70,-3.54,118.97,83.84,7.38
2,new norfolk,12,LR,1604279628,97,6.65,-10.59,74.08,2.59
3,saint-joseph,90,RE,1604279755,56,-21.37,55.62,69.80,5.82
4,hermanus,80,ZA,1604279592,81,-34.42,19.23,64.00,10.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure GMAPS 
gmaps.configure(api_key = g_key)

In [4]:
# Create variables to store data
locations = readcsv[["Lat","Lng"]]
weight = readcsv["Humidity"]

In [6]:
# Create heatmap 
fig = gmaps.figure(center=(20,1), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations,weight, dissipating = False, max_intensity = 300, point_radius = 6)

# Add heat layer to fig 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Clean data to show ideal vacation locations
ideal_cities = readcsv.loc[(readcsv["Max Temp"] > 65) & (readcsv["Max Temp"] < 90)]
ideal_cities = ideal_cities.loc[(readcsv["Wind Speed"] < 10)]
ideal_cities = ideal_cities.loc[(readcsv["Cloudiness"] == 0)]
ideal_cities = ideal_cities.dropna()
ideal_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,hobart,0,AU,1604279661,56,-42.88,147.33,73.99,9.17
66,kaya,0,BF,1604279774,21,13.09,-1.08,78.94,6.91
83,ouahigouya,0,BF,1604279779,16,13.58,-2.42,80.65,9.26
91,segou,0,ML,1604279783,19,13.43,-6.22,77.05,4.70
102,kolondieba,0,ML,1604279786,39,11.09,-6.89,75.34,2.73


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create data frame from ideal cities 
hotel_df = ideal_cities.loc[:, ["City", "Country","Lat","Lng"]]

# Add hotel name to data frame 
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
25,hobart,AU,-42.88,147.33,
66,kaya,BF,13.09,-1.08,
83,ouahigouya,BF,13.58,-2.42,
91,segou,ML,13.43,-6.22,
102,kolondieba,ML,11.09,-6.89,


In [14]:
# Create URL and parameters to call hotel api 
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {
    "radius" : 5000,
    "type" : "hotel",
    "key" : g_key
}

# Store hotels in list
hotel_list = []

for index,row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"] = f"{lat}, {lng}"
    hotels = requests.get(url, params = parameters).json()
    
    try :
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
        print(f"Closest hotel is {hotels['results'][0]['name']} ")
    except:
        print("No hotel found")                                    

Closest hotel is Hobart 
Closest hotel is Kaya 
Closest hotel is Ouahigouya 
Closest hotel is Segou 
Closest hotel is Kolondieba 
Closest hotel is Pokhara 
Closest hotel is Canindé 
Closest hotel is Kavaratti 
Closest hotel is Misrata 
Closest hotel is San 
Closest hotel is Karwar 
Closest hotel is Ouro Sogui 
Closest hotel is Mecca 
Closest hotel is San 
Closest hotel is Arlit 
Closest hotel is Kieta 
Closest hotel is Buldana 
No hotel found
Closest hotel is Salalah 
Closest hotel is Buldana 
Closest hotel is San 
Closest hotel is Oussouye 
Closest hotel is Berbera 
Closest hotel is Buldana 
Closest hotel is Buldana 
Closest hotel is Bosaso 
Closest hotel is Jieyang 
Closest hotel is Buldana 
Closest hotel is Sur 
Closest hotel is Morondava 
Closest hotel is San 
Closest hotel is San Miguel de Tucumán 
Closest hotel is Tonantins 


In [11]:
# Display hotel data frame 
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
25,hobart,AU,-42.88,147.33,Hobart
66,kaya,BF,13.09,-1.08,Kaya
83,ouahigouya,BF,13.58,-2.42,Ouahigouya
91,segou,ML,13.43,-6.22,Segou
102,kolondieba,ML,11.09,-6.89,Kolondieba


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))